In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from config.config import BASE_URL, OUTPUT_FILE
from logger.logger import logger

headers = {'User-Agent': 'Mozilla/5.0'}

def get_unique_item_ids(max_pages=100):
    """Fetch unique item IDs from paginated pages with proper logging."""
    item_ids = set()
    page = 1
    has_more_pages = True
    
    logger.info(f"Starting to scrape item IDs (max pages: {max_pages})")
    
    while has_more_pages:
        if max_pages and page > max_pages:
            logger.info(f"Reached max pages limit ({max_pages})")
            break
            
        url = f"{BASE_URL}?page={page}"
        logger.debug(f"Fetching page {page}: {url}")
        
        try:
            response = requests.get(url, headers=headers)
            
            if response.status_code != 200:
                logger.error(f"Failed to fetch page {page}. Status code: {response.status_code}")
                break

            soup = BeautifulSoup(response.text, "html.parser")
            
            # Find all item links and extract IDs from hrefs
            current_page_ids = set()
            for a in soup.select("a.products-i__link"):
                href = a.get("href")
                if href:
                    match = re.search(r'/autos/(\d+)', href)
                    if match:
                        item_id = match.group(1)
                        current_page_ids.add(item_id)
            
            # Check if we found any new items on this page
            if not current_page_ids:
                logger.info(f"No items found on page {page}, stopping pagination")
                has_more_pages = False
            else:
                new_items = current_page_ids - item_ids
                if not new_items:
                    logger.info(f"No new items found on page {page}, stopping pagination")
                    has_more_pages = False
                else:
                    logger.info(f"Found {len(new_items)} new items on page {page}")
                    item_ids.update(current_page_ids)
                    page += 1
                    
        except requests.RequestException as e:
            logger.error(f"Network error fetching page {page}: {str(e)}", exc_info=True)
            break
        except Exception as e:
            logger.error(f"Unexpected error processing page {page}: {str(e)}", exc_info=True)
            break
    logger.info(f"Finished scraping. Found {len(item_ids)} unique items")
    return item_ids    

# Run the function and print results
item_ids = get_unique_item_ids()
links = list(item_ids)
print(links)
print(len(links))

2025-06-06 22:38:30 - scraper - INFO - Starting to scrape item IDs (max pages: 100)
2025-06-06 22:38:31 - scraper - INFO - Found 36 new items on page 1
2025-06-06 22:38:33 - scraper - INFO - Found 36 new items on page 2
2025-06-06 22:38:34 - scraper - INFO - Found 36 new items on page 3
2025-06-06 22:38:35 - scraper - INFO - Found 36 new items on page 4
2025-06-06 22:38:37 - scraper - INFO - Found 36 new items on page 5
2025-06-06 22:38:38 - scraper - INFO - Found 35 new items on page 6
2025-06-06 22:38:40 - scraper - INFO - Found 36 new items on page 7
2025-06-06 22:38:41 - scraper - INFO - Found 36 new items on page 8
2025-06-06 22:38:43 - scraper - INFO - Found 35 new items on page 9
2025-06-06 22:38:44 - scraper - INFO - Found 36 new items on page 10
2025-06-06 22:38:45 - scraper - INFO - Found 35 new items on page 11
2025-06-06 22:38:47 - scraper - INFO - Found 34 new items on page 12
2025-06-06 22:38:49 - scraper - INFO - Found 34 new items on page 13
2025-06-06 22:38:50 - scrape

['9484384', '9510860', '9351100', '9484758', '9441324', '9052983', '8578868', '8788480', '9516909', '9024938', '9517111', '9512660', '9459764', '9502331', '9470251', '9506084', '9326269', '9450103', '9418318', '9506200', '9490424', '9349674', '9485843', '9515437', '9508341', '9508832', '9481327', '9107332', '9517017', '9392398', '9365914', '9446359', '9516876', '9210688', '9517211', '9441712', '9507062', '9516793', '9508663', '9516714', '9349030', '9004341', '9468488', '9495147', '9510148', '9513197', '9452514', '9454917', '9514828', '9422646', '9283585', '9465227', '9492847', '9317187', '9516785', '9159254', '9504834', '9491660', '9410280', '9517171', '9463209', '9517318', '9500569', '9487048', '9439576', '9491824', '9360799', '9475370', '9486975', '9508512', '9434311', '9342169', '9483275', '9336764', '9513954', '9517337', '9514809', '8599063', '9517186', '9458855', '9517068', '9055786', '9516869', '9433361', '9488112', '8982851', '9452402', '9412171', '9511852', '9517022', '8931169'

In [2]:
def extract_item_data(link):
    """Extract detailed item data from individual product page with comprehensive logging"""
    url = "https://turbo.az/autos/" + link
    logger.debug(f"Starting to extract data from: {url}")
    
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code != 200:
            logger.error(f"Failed to fetch {url} - Status code: {response.status_code}")
            return None

        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract item ID
        item_id = link.split("/")[-1]
        logger.debug(f"Processing item ID: {item_id}")

        # Extract amount and currency using meta tags
        meta_tag = soup.find('meta', {'property': 'og:title'})

        # Extract content attribute
        content = meta_tag.get('content')

        # Use regex to find price followed by either USD or AZN (case-insensitive)
        match = re.search(r'(\d[\d\s]*\d)\s+(USD|AZN)', content, re.IGNORECASE)

        if match:
            amount = match.group(1).replace(' ', '')  # Remove space
            currency = match.group(2).upper()  # Get the currency type

        logger.debug(f"Extracted price: {amount} {currency}")

        # Extract property fields dynamically
        properties = {}
        prop_div = soup.find("div", class_="product-properties__column")
        if prop_div:
            for prop in prop_div.find_all("div", class_="product-properties__i"):
                try:
                    name = prop.find("label", class_="product-properties__i-name").get_text(strip=True)
                    value = prop.find("span", class_="product-properties__i-value").get_text(strip=True)
                    properties[name] = value
                    logger.debug(f"Found property: {name} = {value}")
                except Exception as e:
                    logger.warning(f"Failed to extract property: {str(e)}", exc_info=True)

        # Construct result dictionary
        item_data = {
            'item_id': item_id,
            'amount': amount,
            'currency': currency,
        }

        # Add all found properties dynamically
        item_data.update(properties)
        
        logger.info(f"Successfully processed item: {item_id}")
        return item_data

    except requests.RequestException as e:
        logger.error(f"Network error while processing {url}: {str(e)}", exc_info=True)
        return None
    except Exception as e:
        logger.error(f"Unexpected error processing {url}: {str(e)}", exc_info=True)
        return None


def scrape_all_items():
    """Main scraping function that processes all items with progress tracking"""
    if not links:
        logger.warning("No links to process - empty list provided")
        return

    results = []
    logger.info(f"Starting to process {len(links)} items")

    for idx, link in enumerate(links, 1):
        logger.info(f"[{idx}/{len(links)}] Processing: {link}")
        data = extract_item_data(link)
        if data:
            results.append(data)
            logger.debug(f"Added item {data.get('item_id')} to results")
        else:
            logger.warning(f"Failed to process item: {link}")

    # Save to JSON
    try:
        with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
            json.dump(results, f, indent=2, ensure_ascii=False)
        logger.info(f"Successfully saved {len(results)} items to {OUTPUT_FILE}")
        if len(results) != len(links):
            logger.warning(f"Processed {len(results)} out of {len(links)} items ({(len(results)/len(links))*100:.1f}% success rate)")
    except Exception as e:
        logger.error(f"Failed to save data to {OUTPUT_FILE}: {str(e)}", exc_info=True)


# Run the scraper
scrape_all_items()

2025-06-06 22:42:14 - scraper - INFO - Starting to process 3108 items
2025-06-06 22:42:14 - scraper - INFO - [1/3108] Processing: 9484384
2025-06-06 22:42:15 - scraper - INFO - Successfully processed item: 9484384
2025-06-06 22:42:15 - scraper - INFO - [2/3108] Processing: 9510860
2025-06-06 22:42:15 - scraper - INFO - Successfully processed item: 9510860
2025-06-06 22:42:15 - scraper - INFO - [3/3108] Processing: 9351100
2025-06-06 22:42:16 - scraper - INFO - Successfully processed item: 9351100
2025-06-06 22:42:16 - scraper - INFO - [4/3108] Processing: 9484758
2025-06-06 22:42:17 - scraper - INFO - Successfully processed item: 9484758
2025-06-06 22:42:17 - scraper - INFO - [5/3108] Processing: 9441324
2025-06-06 22:42:17 - scraper - INFO - Successfully processed item: 9441324
2025-06-06 22:42:17 - scraper - INFO - [6/3108] Processing: 9052983
2025-06-06 22:42:18 - scraper - INFO - Successfully processed item: 9052983
2025-06-06 22:42:18 - scraper - INFO - [7/3108] Processing: 857886